In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from mediaviz.draw import draw_forceatlas2_network
from mediaviz.viz_parser import parse_colors, parse_size
from itertools import combinations

from community import community_louvain
from networkx.algorithms.clique import find_cliques as maximal_cliques
import networkx as nx
from networkx.algorithms import bipartite
import sqlite3 
import sys
import time
import uuid
import pandas as pd
import json
import numpy
count_group=0

def convert(o):
    if isinstance(o, numpy.int64): return int(o)  
    raise TypeError



#open json file
output  = open("retweeter_group_alhilal.json","w")


#sqliteConnection = sqlite3.connect('Retweet_manipulation.db')
#cursor = sqliteConnection.cursor()
#G = nx.erdos_renyi_graph(30, 0.05)
#community louvain
#base_retweeter_graph = nx.read_weighted_edgelist('test_graph1.weighted.edgelist',nodetype=int)
#partition = community_louvain.best_partition(base_retweeter_graph)
#print(set (partition.values()))


dict_partition = {row[0] : row[1] for _, row in pd.read_csv("partitions_alhilal.csv").iterrows()}

for com in set(dict_partition.values()):
    #count_group+=1
    bipartite_graph = nx.Graph()
    list_nodes_retweeter=[]
    list_nodes_retweeter = [nodes for nodes in dict_partition.keys()if dict_partition[nodes] == com]
    
    
    #set_retweeter.extend(list_nodes)
    #print(set_retweeter)
    bipartite_graph.add_nodes_from(list_nodes_retweeter, bipartite=0)
    #print (tuple(set_retweeter))
    
    tuple_set_retweeter=tuple(list_nodes_retweeter)
    #print(tuple_set_retweeter)
    with sqlite3.connect('Retweet_manipulation3.db') as conn:
        cursor = conn.cursor()    
        """
        cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
        retweeter_tweet_ids_records = cursor.fetchall()
        set_tweet=[row[1] for row in retweeter_tweet_ids_records]
        bipartite_graph.add_nodes_from(set_tweet, bipartite=1)
        """
        cursor.execute("SELECT tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
        
        while True:
            tweet_id_record = cursor.fetchone()  
            
            if tweet_id_record==None:
                break
            bipartite_graph.add_node(tweet_id_record[0], bipartite=1)
        
        """
        set_edges=[]
        for row in retweeter_tweet_ids_records:
            set_edges.append((row[0], row[1]))
    
        bipartite_graph.add_edges_from(set_edges)
        """
        cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
        while True:
            ret_tw_ids_record  = cursor.fetchone()  
            
            if ret_tw_ids_record==None:
                break
            bipartite_graph.add_edge(ret_tw_ids_record[0],ret_tw_ids_record[1])
    
    retweeter_nodes = {n for n, d in bipartite_graph.nodes(data=True) if d['bipartite']==0}
    candidate_retweeter_group = bipartite.weighted_projected_graph(bipartite_graph, retweeter_nodes,ratio=False)
    
    #drop all edges has less thah 3 to remove inactive user from entire group
    edges_less_weight = [(u, v,d) for (u, v, d) in candidate_retweeter_group.edges(data=True) if d['weight'] < 2 ]
    candidate_retweeter_group.remove_edges_from(edges_less_weight)
    candidate_retweeter_group.remove_nodes_from(list(nx.isolates(candidate_retweeter_group)))
    
    #print(Get_retweeter_groups.edges.data() )
    #count_group+=1
    
    #print("new group:  ",count_group)
    #print(list( nx.connected_components(candidate_retweeter_group)))
    #print("\n")
    list_connected_component=[]
    list_connected_component=list(c for c in nx.connected_components(candidate_retweeter_group) if len(c)>2)
    for cc in list_connected_component:
    #for cc in list( nx.connected_components(candidate_retweeter_group)):
        #print(cc)
        #print("\n")
        #connected_component= Get_retweeter_groups.subgraph(cc).copy()
        connected_component= candidate_retweeter_group.subgraph(cc)
         #[G.subgraph(c).copy() for c in connected_components(G)]
        
        #nx.draw(subgraph,with_labels=True)
        #plt.show()
        #path= 'graphname.gexf'
        #draw_forceatlas2_network(connected_component,node_color='purple',with_labels=True, adjust_labels=True, node_size=10, edge_color='gray')
        cliques = sorted (list(frozenset(c) for c in nx.find_cliques(connected_component)  ) , key=len, reverse=True)
        
        #remove overlapping cliques
        for c1, c2 in combinations(cliques, 2):
            #print (c1,c2)
            if len(c1.intersection(c2)) >0:
                if c2 in cliques:
                    cliques.remove(c2)
        #print(cliques)
        #sub_group=0
        
        #print(cliques)
        #print('\n')
        for cl in cliques:
            #sub_group+=1
            #print ("sub group: ", sub_group)
            cur_community=list(cl)
            neighborList=[]
            growing_threshold=len(cur_community) * 0.7
            #print(cur_community)
            for node in cur_community:
                neighborList.extend(list(n for n in connected_component.neighbors(node)))
                
                #print(neighborList)
            update_neighborList=set (neighborList).difference(set(cur_community))
            #print(update_neighborList)
            growingList = cur_community
            for n in list(update_neighborList):
                #print(n)
                incomingEdgeCount=set(cur_community).intersection(set(n for n in connected_component.neighbors(n)))
                if len(incomingEdgeCount)>= growing_threshold:
                    growingList.append(n)
            #saveing new cur_community to database        
            #print (growingList)
            #count_group+=1
            #print (count_group)
            
            
            
            #save id of retweeter group in json file
            
            id_group = uuid.uuid4()
            json_data = {}
            json_data["group_id"]=id_group.hex
            json_data["members_id"]=growingList
            #json.dumps({'value': numpy.int64(42)}, default=convert)
            json.dump(json_data, output, default=convert)
            #json.dump(json_data, output)
            output.write('\n')
            """
            try:
                json.dump(json_data, output)
                output.write('\n')
            finally:
                output.close()   
            """
            
            #sub = connected_component.subgraph(growingList)
            
            #count_group+=1
            """
            graph_information = pd.DataFrame(sub.edges.data())
            graph_information2 = pd.DataFrame(columns = ["source", "target","type" ,"weight"])
            #for edge in retweeter_graph.edges.data():
      
            graph_information2["source"]=graph_information[0].apply(lambda x: x)
            graph_information2["target"]=graph_information[1].apply(lambda x: x)
            graph_information2["weight"]=graph_information[2].apply(lambda x: x['weight'])
            graph_information2["type"]="Undirected"
            #file_name='%s_%s.csv' % ('Network_structure_group', time.strftime('%Y%m%d-%H%M%S'))
            file_name='%s_%s.csv' % ('Network_structure_group', count_group)
            graph_information2.to_csv(file_name,index=True)
            
            """
            #draw_forceatlas2_network(sub,node_color='purple',with_labels=True, adjust_labels=True, node_size=10, edge_color='gray')
            #sub_group+=1
            
            #count_group+=1
            #network="Network structure of group: "
            #draw_forceatlas2_network(sub,node_color='purple', title="%s%s" % (network, count_group), edge_color='black')
            
            #nx.draw(sub,with_labels=True)  
            #plt.show()
            
    #Cleaning the Garbage        
    bipartite_graph.clear()
            
output.close()             

KeyboardInterrupt: 

In [1]:
import networkx as nx
base_retweeter_graph = nx.read_weighted_edgelist('hashtage_alhilal.weighted.edgelist',nodetype=int)

#partition = community_louvain.best_partition(base_retweeter_graph)

In [2]:
from community import community_louvain
partition = community_louvain.best_partition(base_retweeter_graph)
print(len(set (partition.values())))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-3480596e4365>", line 2, in <module>
    partition = community_louvain.best_partition(base_retweeter_graph)
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\community\community_louvain.py", line 249, in best_partition
    random_state)
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\community\community_louvain.py", line 343, in generate_dendrogram
    current_graph = graph.copy()
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\networkx\classes\graph.py", line 1515, in copy
    for u, nbrs in self._adj.items()
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\networkx\classes\graph.py", line 932, in add_edges_from
    for e in ebunch_to_add:
  File "C:\Users\Welcome\Anaconda3\lib\site-packages\networkx\classes\graph.py", line 1516, in <genexpr>
    f

TypeError: can only concatenate str (not "list") to str